## Comparing the initial mass function (IMF)

This code will compare the mass function depending on the selection of the three IMFs: [Salpeter (1955)](http://articles.adsabs.harvard.edu/pdf/1955ApJ...121..161S), [Kroupa (2001)](http://articles.adsabs.harvard.edu/pdf/2001MNRAS.322..231K), and [Chabrier (2003)](https://iopscience.iop.org/article/10.1086/376392/pdf)

### Packages

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate

In [2]:
# %matplotlib notebook
%matplotlib widget

### The three IMFs

The IMF formula look like the below:

$dn=\xi(m)dm$    
($\xi(m)$: the mass function, $m$: the mass in the unit of solar mass)

* Salpeter IMF: $\xi(m)=\xi_{0}m^{-2.35}$

* Kroupa IMF: $\xi(m)=\xi_{0}m^{-\alpha}$

    $\alpha=0.3$ for $m<0.08$    
    $\alpha=1.3$ for $m=0.08-0.5$    
    $\alpha=2.3$ for $m>0.5$

* Chabrier IMF:    
    For $m<1$, $\xi(m)=0.086\times\frac{1}{m~\ln10}\times\exp{[-\frac{(\log m-\log 0.22)^{2}}{2\times0.57^{2}}]}$    
    For $m>1$, $\xi(m)=\xi_{0}m^{-2.3}$

### Normalizations of the IMFs
* Intergration range: ($0.01~M_{\odot}-\infty$)

In [3]:
Mmin, Mmax = 0.01, np.inf

In [4]:
# ----- Normalizing the Salpeter IMF ----- #
y, abserr = integrate.quad(lambda m: m**(-2.35), Mmin, Mmax)
print(y, abserr)
print(f"k = {1. / y:.4e}")
k_Salpeter = 1. / y

371.2498026868661 4.2672672861954197e-07
k = 2.6936e-03


In [5]:
# ----- Normalizing the Kroupa IMF ----- #

# Range 1: [0.50 - inf]
y1, abserr = integrate.quad(lambda m: m**(-2.3), 0.50, Mmax)
print(y1, abserr)

# Range 2: [0.08 - 0.50]
M1 = 0.50
Y1 = M1**(-2.3)
y2, abserr = integrate.quad(lambda m: Y1*(m/M1)**(-1.3), 0.08, 0.50)
print(y2, abserr)

# Range 3: [0.01 - 0.08]
M2 = 0.08
Y2 = Y1*(M2/M1)**(-1.3)
y3, abserr = integrate.quad(lambda m: Y2*(m/M2)**(-0.3), Mmin, 0.08)
print(y3, abserr)

print(f"k = {1. / (y1+y2+y3):.4e}")
k_Kroupa = 1. / (y1+y2+y3)

1.894068328222913 8.565341547139838e-09
6.0150605991888355 1.0211438118605483e-10
4.673627257232486 1.4483491636472381e-11
k = 7.9474e-02


In [6]:
# ----- Normalizing the Chabrier IMF ----- #
y = lambda m: 0.086/(m*np.log(10.0)) * np.exp(-(np.log10(m)-np.log10(0.22))**2 / (2.0*0.57**2))

# Range 1: [1.0 - inf]
y1, abserr = integrate.quad(lambda m: m**(-2.3), 1.0, Mmax)
print(y1, abserr)

# Range 2: [0.01 - 1.0]
M1 = 1.0
Y1 = M1**(-2.3)
y2, abserr = integrate.quad(lambda m: (Y1/y(M1))*y(m), Mmin, 1.0)
print(y2, abserr)

print(f"k = {1. / (y1+y2):.4e}")
k_Chabrier = 1. / (y1+y2)

0.7692307692307694 8.540177112501205e-16
5.545090488390361 9.288481782370129e-10
k = 1.5837e-01


### The definition of the IMFs

In [7]:
# ----- Function definition ----- #

def Salpeter_IMF(m):
    k = k_Salpeter
    alpha = 2.35
    return k*m**(-alpha)


def Kroupa_IMF(m, Mmin=Mmin):
    k = k_Kroupa
    M1 = 0.50
    Y1 = M1**(-2.3)
    M2 = 0.08
    Y2 = Y1*(M2/M1)**(-1.3)
    
    if np.isscalar(m):
        if ((m >= Mmin) & (m <= 0.08)):
            val = k*Y2*(m/M2)**(-0.3)
        elif ((m > 0.08) & (m <= 0.50)):
            val = k*Y1*(m/M1)**(-1.3)
        else:
            val = k*m**(-2.3)
            
    else:
        cnd1 = (m >= Mmin) & (m <= 0.08)
        cnd2 = (m > 0.08) & (m <= 0.50)
        cnd3 = (m > 0.50)
        val = np.zeros(m.size)
        val[cnd1] = k*Y2*(m[cnd1]/M2)**(-0.3)
        val[cnd2] = k*Y1*(m[cnd2]/M1)**(-1.3)
        val[cnd3] = k*m[cnd3]**(-2.3)
    
    return val


def lm_imf(m):
    return 0.086/(m*np.log(10.0)) * np.exp(-(np.log10(m)-np.log10(0.22))**2 / (2.0*0.57**2))

def Chabrier_IMF(m, Mmin=Mmin):
    k = k_Chabrier
    alpha = 2.3
    M1 = 1.0
    Y1 = M1**(-2.3)
    v0 = lm_imf(m)
    
    if np.isscalar(m):
        if ((m >= Mmin) & (m <= 1.0)):
            val = k*(Y1/lm_imf(M1))*v0
        else:
            val = k*m**(-alpha)
    
    else:
        cnd1 = (m >= Mmin) & (m <= 1.0)
        cnd2 = (m > 1.0)
        val = np.zeros(m.size)
        val[cnd1] = k*(Y1/lm_imf(M1))*v0[cnd1]
        val[cnd2] = k*m[cnd2]**(-alpha)
        
    return val

In [8]:
# Checking the integrals
y, abserr = integrate.quad(Salpeter_IMF, Mmin, Mmax)
print(y, abserr)

y, abserr = integrate.quad(Kroupa_IMF, Mmin, Mmax)
print(y, abserr)

y, abserr = integrate.quad(Chabrier_IMF, Mmin, Mmax)
print(y, abserr)

1.0000000000000007 1.1494314389182136e-09
0.9999999997606237 4.528815100712791e-10
1.0000000003645921 1.447007658139654e-08


### Plotting the IMFs

In [9]:
m_arr = np.linspace(0.01, 500, 100000)
m_arr

array([1.000000e-02, 1.499995e-02, 1.999990e-02, ..., 4.999900e+02,
       4.999950e+02, 5.000000e+02])

In [10]:
fig, ax = plt.subplots()
p1, = ax.loglog(m_arr, Salpeter_IMF(m_arr)/k_Salpeter, label="Salpeter (1955) IMF")
p2, = ax.loglog(m_arr, Kroupa_IMF(m_arr)/k_Kroupa, label="Kroupa (2001) IMF")
p3, = ax.loglog(m_arr, Chabrier_IMF(m_arr)/k_Chabrier, label="Chabrier (2003) IMF")
ax.set_xlim([1.0e-2, 1.0e+2])
ax.set_ylim([1.0e-3, 1.0e+3])
ax.set_xlabel(r"Mass [$M_{\odot}$]")
ax.set_ylabel("Mass function (not normalized)")
ax.legend()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fig, ax = plt.subplots()
p1, = ax.loglog(m_arr, Salpeter_IMF(m_arr), label="Salpeter (1955) IMF")
p2, = ax.loglog(m_arr, Kroupa_IMF(m_arr), label="Kroupa (2001) IMF")
p3, = ax.loglog(m_arr, Chabrier_IMF(m_arr), label="Chabrier (2003) IMF")
ax.set_xlim([1.0e-2, 1.0e+2])
ax.set_ylim([1.0e-3, 1.0e+3])
ax.set_xlabel(r"Mass [$M_{\odot}$]")
ax.set_ylabel("Mass function (normalized)")
ax.legend()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Integrals with respect to mass

In [12]:
# Checking the integrals
i = 0
for f in [Salpeter_IMF, Kroupa_IMF, Chabrier_IMF]:
    i += 1
    M, abserr = integrate.quad(lambda m: m * f(m), 0.1, 100)
    L, abserr = integrate.quad(lambda m: m**3 * f(m), 0.1, 100)
    print(f"M/L = {M/L:.3e}")
    exec(f"ml_{i:d} = M/L")

M/L = 4.818e-03
M/L = 3.015e-03
M/L = 2.612e-03


In [13]:
print(f"M/L (Salpeter) = {ml_1/ml_2:.3f} M/L (Kroupa)")
print(f"M/L (Salpeter) = {ml_1/ml_3:.3f} M/L (Chabrier)")
print(f"M/L (Kroupa) = {ml_2/ml_1:.3f} M/L (Salpeter)")
print(f"M/L (Chabrier) = {ml_3/ml_1:.3f} M/L (Salpeter)")

M/L (Salpeter) = 1.598 M/L (Kroupa)
M/L (Salpeter) = 1.845 M/L (Chabrier)
M/L (Kroupa) = 0.626 M/L (Salpeter)
M/L (Chabrier) = 0.542 M/L (Salpeter)
